<a href="https://colab.research.google.com/github/SeaMoose6/MathMex/blob/master/hugging_face_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers datasets
! pip install transformers sentencepiece -q
! pip install wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=7b2dada109e61123336b762f0be9843fa5df12ed1106e481b2a1486ef995ebfe
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
from transformers import pipeline
import wikipedia
import sys
import requests
import csv


In [15]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image

In [ ]:
pipe = pipeline(task='text2text-generation', model='facebook/m2m100_418M')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [ ]:
def return_languages(topic):
  # print(wikipedia.languages())
  sum_list = []
  wikipedia.set_lang('en')
  # print(wikipedia.summary(topic))
  sum_list.append(wikipedia.summary(topic))
  wikipedia.set_lang('es')
  translation = pipe(topic, forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='es'))
  es_name = translation[0]['generated_text']
  try:
    sum_list.append(wikipedia.summary(es_name))
  except:
    sum_list.append('page not found')
  wikipedia.set_lang('fr')
  fr_translation = pipe(topic, forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='fr'))
  fr_name = fr_translation[0]['generated_text']
  try:
    sum_list.append(wikipedia.summary(fr_name))
  except:
    sum_list.append('page not found')
  wikipedia.set_lang('zh')
  zh_translation = pipe(topic, forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='zh'))
  zh_name = zh_translation[0]['generated_text']
  try:
    sum_list.append(wikipedia.summary(zh_name))
  except:
    sum_list.append('page not found')
  wikipedia.set_lang('fa')
  fa_translation = pipe(topic, forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='fa'))
  fa_name = fa_translation[0]['generated_text']
  try:
    sum_list.append(wikipedia.summary(fa_name))
  except:
    sum_list.append('page not found')
  return sum_list
  print(sum_list)
# return_languages("Gini coefficient")
return_languages('Srinivasa Ramanujan')


In [ ]:
# url = "https://th.bing.com/th/id/OIP.EJsikDvg2o3jjJkfyYsubwHaFj?w=248&h=186&c=7&r=0&o=5&pid=1.7"
# processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
# model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

# inputs = processor(images=image, return_tensors="pt")
# outputs = model(**inputs)

# target_sizes = torch.tensor([image.size[::-1]])
# results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]
# for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
#     box = [round(i, 2) for i in box.tolist()]
#     print(
#             f"Detected {model.config.id2label[label.item()]} with confidence "
#             f"{round(score.item(), 3)} at location {box}"
#     )


In [34]:
#with open('English_wiki.tsv', 'w', encoding='utf-8') as f:
import csv
def read_title(file_path):
  lst_titles = []
  with open(file_path) as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
      lst_titles.append(row[0])
  return lst_titles[:10:2]
lst_titles = read_title("wiki_origin.tsv")
print(lst_titles)

['Gini coefficient', 'Thermal conductivity', 'Srinivasa Ramanujan', 'Embedding', 'Income']


In [38]:
def append_data(title_list):
  for title in title_list:
      translations = return_languages(title)
      with open("english_data.tsv", 'w') as ed:
        ed.write(translations[0] + '\n')
      with open("chinese_data.tsv", 'w') as sd:
        sd.write(translations[3] + '\n')
      with open("french_data.tsv", 'w') as sd:
        sd.write(translations[2] + '\n')
      with open("persian_data.tsv", 'w') as sd:
        sd.write(translations[4] + '\n')

lst_titles = read_title("wiki_origin.tsv")
append_data(lst_titles)


/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


PageError: Page id "in com" does not match any pages. Try another id!